In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

# local module
import pre_processing_funcs as pre

In [2]:
files = glob("data/Train_Batch_*.csv")
files

['data\\Train_Batch_1.csv',
 'data\\Train_Batch_2.csv',
 'data\\Train_Batch_3.csv']

In [3]:
dfs = [pd.read_csv(file) for file in files]
print(dfs[0].shape)
print(dfs[1].shape)
print(dfs[2].shape)

(19440, 17)
(12956, 18)
(6466, 19)


There is a problem we can't concat our data because files have different columns we need to pre-process every single file alone then concat them!

In [4]:
df1 = dfs[0].copy()
df2 = dfs[1].copy()
df3 = dfs[2].copy()

In [5]:
print(df1.shape)
print(df1.info())
df1.head(2)

(19440, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19440 entries, 0 to 19439
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     19440 non-null  object 
 1   Person Description             19440 non-null  object 
 2   Place Code                     19440 non-null  object 
 3   Customer Order                 19440 non-null  object 
 4   Additional Features in market  15140 non-null  object 
 5   Promotion Name                 16530 non-null  object 
 6   Store Kind                     16530 non-null  object 
 7   Store Sales                    19440 non-null  object 
 8   Store Cost                     19440 non-null  object 
 9   Product Weights Data in (KG)   19440 non-null  object 
 10  Is Recyclable?                 19440 non-null  object 
 11  Min. Yearly Income             19437 non-null  object 
 12  Store Area                     174

,Unnamed: 0,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost
0,mc_ID_0,"Single Female with four children, education: b...",H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,323,602.7575
1,mc_ID_1,"Single Female with three children, education: ...",S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,NaN,708.6650


change id table name and set index to id

In [6]:
df1 = pre.set_index(df1)
df1.head(2)

,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost
id,,,,,,,,,,,,,,,,
mc_ID_0,"Single Female with four children, education: b...",H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,323,602.7575
mc_ID_1,"Single Female with three children, education: ...",S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,NaN,708.6650


In [7]:
df1 = pre.split_person_description(df1)
df1.head(2)

,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,Marriage,Gender,Children,Degree,Work
id,,,,,,,,,,,,,,,,,,,,
mc_ID_0,H11go_ZA,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,323,602.7575,Single,Female,four,bachelors degree,professional
mc_ID_1,S04ne_WA,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,NaN,708.6650,Single,Female,three,bachelors degree,management


In [8]:
df1["Place Code"].value_counts()

T02ma_WA    1802
S01em_OR    1771
P07nd_OR    1668
S03le_WA    1626
H11go_ZA    1508
M10da_YU    1452
S04ne_WA    1411
B06ls_CA    1355
L05es_CA    1322
B14on_WA    1109
V13er_BC    1073
O09ba_VE     861
C15ho_ZA     769
A17co_GU     503
M12ty_DF     429
S18co_CA     253
B08am_WA     228
V19ia_BC     177
G16ra_JA     123
Name: Place Code, dtype: int64

In [9]:
df1 = pre.split_place_code(df1)
df1.head()

,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,...,Frozen Area,Meat Area,Cost,Marriage,Gender,Children,Degree,Work,Store Code,Country ISO2
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,"Cleaning Supplies from Household department, O...","['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,...,481.98,323,602.7575,Single,Female,four,bachelors degree,professional,H11go,ZA
mc_ID_1,"Snack Foods from Snack Foods department, Order...",NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,...,457.36,NaN,708.6650,Single,Female,three,bachelors degree,management,S04ne,WA
mc_ID_2,"Magazines from Periodicals department, Ordered...",['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,"{'Gross Weight': 28.6358, 'Net Weight': 27.182...",recyclable,30K+,2192.32,...,523.32,348.85,564.2647,Married,Male,two,high school degree,skilled manual,L05es,CA
mc_ID_3,"Dairy from Dairy department, Ordered Brand : C...",['Coffee Bar'],Savings Galore,NaN,1.98 Millions,0.6732 Millions,"{'Gross Weight': 14.2161, 'Net Weight': 11.294...",recyclable,30K+,1974.73,...,440.92,293.95,724.5119,Single,Female,three,partial college,professional,S03le,WA
mc_ID_4,"Vegetables from Produce department, Ordered Br...","['Bar For Salad', 'Ready Food', 'Coffee Bar', ...",Sale Winners,Deluxe,11.56 Millions,4.9708 Millions,"{'Gross Weight': 12.6172, 'Net Weight': 9.71, ...",non recyclable,50K+,2862.3,...,593.93,395.95,519.7574,Married,Female,five,bachelors degree,professional,M10da,YU


In [10]:
df1 = pre.split_customer_order(df1)
df1.head(3)

,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,...,Marriage,Gender,Children,Degree,Work,Store Code,Country ISO2,Order Brand,Order,Department
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,"['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,...,Single,Female,four,bachelors degree,professional,H11go,ZA,Red Wing,Cleaning Supplies,Household
mc_ID_1,NaN,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,...,Single,Female,three,bachelors degree,management,S04ne,WA,Nationeel,Snack Foods,Snack Foods
mc_ID_2,['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,"{'Gross Weight': 28.6358, 'Net Weight': 27.182...",recyclable,30K+,2192.32,1322.21,...,Married,Male,two,high school degree,skilled manual,L05es,CA,Excel,Magazines,Periodicals


In [11]:
df1 = pre.encode_market_features(df1)
df1.head(3)

,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,...,Store Code,Country ISO2,Order Brand,Order,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8.76 Millions,4.2924 Millions,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,...,H11go,ZA,Red Wing,Cleaning Supplies,Household,1,1,1,1,1
mc_ID_1,Budget Bargains,Supermarket,6.36 Millions,1.9716 Millions,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,...,S04ne,WA,Nationeel,Snack Foods,Snack Foods,0,0,0,0,0
mc_ID_2,Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,"{'Gross Weight': 28.6358, 'Net Weight': 27.182...",recyclable,30K+,2192.32,1322.21,523.32,...,L05es,CA,Excel,Magazines,Periodicals,1,0,0,0,0


In [12]:
df1["Promotion Name"].value_counts()

Save Price                  676
Weekend Discount            660
Two Day Sale                633
Price Winners               579
Super Savers                568
Save It (Sale)              541
One Day Sale                519
Super Duper Savers          508
Roller Savings (High)       497
GLD                         478
Price Slashers              476
Full Free                   463
Shelf Clearing Days         459
Sale : Double Down          457
Lottery Cash Registerion    418
Two for One                 407
Big Time Discounts          400
Go For It                   394
Money Savers                379
Price Destroyers            369
Budget Bargains             360
Saving Days                 351
Discount Frenzy             332
Best Price Savers           327
Price Cutters               323
Dimes Off                   301
Savings Galore              298
Bag Stuffer                 287
Promo Big                   285
You Save Days               283
Sales Days                  279
I Cant B

In [13]:
df1["Store Kind"].value_counts()

Supermarket      7209
Deluxe           6220
Gourmet          1821
Mid-Size          761
Small Grocery     519
Name: Store Kind, dtype: int64

In [14]:
df1 = pre.transform_cost_sales(df1)
df1.head(3)

,Promotion Name,Store Kind,Store Sales,Store Cost,Product Weights Data in (KG),Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,...,Store Code,Country ISO2,Order Brand,Order,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.0,4292400.0,"{'Gross Weight': 28.1997, 'Net Weight': 26.600...",recyclable,10K+,2842.23,2037.64,481.98,...,H11go,ZA,Red Wing,Cleaning Supplies,Household,1,1,1,1,1
mc_ID_1,Budget Bargains,Supermarket,6360000.0,1971600.0,"{'Gross Weight': 16.571, 'Net Weight': 14.972,...",non recyclable,50K+,2814.95,2049.72,457.36,...,S04ne,WA,Nationeel,Snack Foods,Snack Foods,0,0,0,0,0
mc_ID_2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,"{'Gross Weight': 28.6358, 'Net Weight': 27.182...",recyclable,30K+,2192.32,1322.21,523.32,...,L05es,CA,Excel,Magazines,Periodicals,1,0,0,0,0


In [15]:
df1 = pre.extract_product_weights(df1)
df1.head(3)

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,...,Order,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Gross Weight,Net Weight,Package Weight
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.0,4292400.0,recyclable,10K+,2842.23,2037.64,481.98,323,...,Cleaning Supplies,Household,1,1,1,1,1,28.1997,26.6008,1.599
mc_ID_1,Budget Bargains,Supermarket,6360000.0,1971600.0,non recyclable,50K+,2814.95,2049.72,457.36,NaN,...,Snack Foods,Snack Foods,0,0,0,0,0,16.571,14.972,1.599
mc_ID_2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,recyclable,30K+,2192.32,1322.21,523.32,348.85,...,Magazines,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536


In [16]:
df1 = pre.transform_recyclable(df1)
df1.head(3)

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Min. Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,...,Order,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Gross Weight,Net Weight,Package Weight
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.0,4292400.0,yes,10K+,2842.23,2037.64,481.98,323,...,Cleaning Supplies,Household,1,1,1,1,1,28.1997,26.6008,1.599
mc_ID_1,Budget Bargains,Supermarket,6360000.0,1971600.0,no,50K+,2814.95,2049.72,457.36,NaN,...,Snack Foods,Snack Foods,0,0,0,0,0,16.571,14.972,1.599
mc_ID_2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,yes,30K+,2192.32,1322.21,523.32,348.85,...,Magazines,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536


In [17]:
df1 = pre.transform_income(df1)
df1.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,...,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.0,4292400.0,yes,2842.23,2037.64,481.98,323,602.7575,...,Household,1,1,1,1,1,28.1997,26.6008,1.599,10000.0
mc_ID_1,Budget Bargains,Supermarket,6360000.0,1971600.0,no,2814.95,2049.72,457.36,NaN,708.6650,...,Snack Foods,0,0,0,0,0,16.571,14.972,1.599,50000.0
mc_ID_2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,yes,2192.32,1322.21,523.32,348.85,564.2647,...,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536,30000.0
mc_ID_3,Savings Galore,NaN,1980000.0,673200.0,yes,1974.73,NaN,440.92,293.95,724.5119,...,Dairy,0,1,0,0,0,14.2161,11.2944,2.9217,30000.0
mc_ID_4,Sale Winners,Deluxe,11560000.0,4970800.0,no,2862.3,1872.19,593.93,395.95,519.7574,...,Produce,1,1,1,1,1,12.6172,9.71,2.9072,50000.0


In [18]:
df1 = pre.transform_columns_type(df1)
df1.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,...,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.0,4292400.0,yes,2842.23,2037.64,481.98,323.00,602.7575,...,Household,1,1,1,1,1,28.1997,26.6008,1.5990,10000.0
mc_ID_1,Budget Bargains,Supermarket,6360000.0,1971600.0,no,2814.95,2049.72,457.36,NaN,708.6650,...,Snack Foods,0,0,0,0,0,16.5710,14.9720,1.5990,50000.0
mc_ID_2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,yes,2192.32,1322.21,523.32,348.85,564.2647,...,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536,30000.0
mc_ID_3,Savings Galore,NaN,1980000.0,673200.0,yes,1974.73,NaN,440.92,293.95,724.5119,...,Dairy,0,1,0,0,0,14.2161,11.2944,2.9217,30000.0
mc_ID_4,Sale Winners,Deluxe,11560000.0,4970800.0,no,2862.30,1872.19,593.93,395.95,519.7574,...,Produce,1,1,1,1,1,12.6172,9.7100,2.9072,50000.0


In [19]:
df1 = pre.encode_columns(df1)
df1.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,...,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.0,4292400.0,True,2842.23,2037.64,481.98,323.00,602.7575,...,Household,1,1,1,1,1,28.1997,26.6008,1.5990,10000.0
mc_ID_1,Budget Bargains,Supermarket,6360000.0,1971600.0,False,2814.95,2049.72,457.36,NaN,708.6650,...,Snack Foods,0,0,0,0,0,16.5710,14.9720,1.5990,50000.0
mc_ID_2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,True,2192.32,1322.21,523.32,348.85,564.2647,...,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536,30000.0
mc_ID_3,Savings Galore,NaN,1980000.0,673200.0,True,1974.73,NaN,440.92,293.95,724.5119,...,Dairy,0,1,0,0,0,14.2161,11.2944,2.9217,30000.0
mc_ID_4,Sale Winners,Deluxe,11560000.0,4970800.0,False,2862.30,1872.19,593.93,395.95,519.7574,...,Produce,1,1,1,1,1,12.6172,9.7100,2.9072,50000.0


In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19440 entries, mc_ID_0 to mc_ID_19354
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Promotion Name             16530 non-null  object 
 1   Store Kind                 16530 non-null  object 
 2   Store Sales                19440 non-null  float64
 3   Store Cost                 19440 non-null  float64
 4   Is Recyclable?             19440 non-null  bool   
 5   Store Area                 17482 non-null  float64
 6   Grocery Area               17459 non-null  float64
 7   Frozen Area                17506 non-null  float64
 8   Meat Area                  17492 non-null  float64
 9   Cost                       19416 non-null  float64
 10  Marriage                   19440 non-null  object 
 11  Gender                     19440 non-null  object 
 12  Children                   19440 non-null  int32  
 13  Degree                     19440 non-nu

In [21]:
df2.head(3)

,Unnamed: 0,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Is Recyclable?,Min. Person Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,Cost
0,mc_ID_0,"Single Male with two children, education: high...",S01em_OR,"Baking Goods from Baking Goods department, Ord...","['Video Store', 'Florist', 'Ready Food', 'Bar ...",Tip Top Savings,Deluxe,3.28 M,0.984 M,21.9493,19.0421,yes,30K+,2577.50,1734.50,503.07,335.38,287.1270
1,mc_ID_1,"Married Male with four children, education: hi...",T02ma_WA,"Canned Sardines from Canned Foods department, ...","['Florist', 'Bar For Salad', 'Ready Food', 'Co...",Discount Frenzy,Deluxe,9.35 M,4.2075 M,19.1874,16.1349,yes,30K+,3145.51,2057.74,654.13,NaN,779.6884
2,mc_ID_2,"Married Female with three children, education:...",S04ne_WA,"Frozen Desserts from Frozen Foods department, ...",[],Tip Top Savings,Supermarket,8.4 M,3.192 M,16.7163,12.3555,no,30K+,2811.99,2049.72,457.36,304.91,557.8318


In [22]:
df2 = pre.set_index(df2)
df2 = pre.split_person_description(df2)
df2 = pre.split_place_code(df2)
df2 = pre.split_customer_order(df2)
df2 = pre.encode_market_features(df2)
df2 = pre.transform_cost_sales(df2)
df2 = pre.transform_income(df2)
df2 = pre.calculate_package_weight(df2)
#df2 = transform_columns_type(df2)
df2["Frozen Area"] = df2["Frozen Area"][(df2["Frozen Area"] != '""')].str.rstrip('.').astype(float)
df2 = df2.drop(columns="")
df2 = pre.encode_columns(df2)
df2.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Is Recyclable?,Min. Person Yearly Income,Store Area,Grocery Area,...,Country ISO2,Order Brand,Order,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Package Weight
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Tip Top Savings,Deluxe,3280000.0,984000.0,21.9493,19.0421,True,30000.0,2577.50,1734.50,...,OR,Landslide,Baking Goods,Baking Goods,1,1,1,1,1,2.9072
mc_ID_1,Discount Frenzy,Deluxe,9350000.0,4207500.0,19.1874,16.1349,True,30000.0,3145.51,2057.74,...,WA,Pleasant,Canned Sardines,Canned Foods,1,1,1,0,1,3.0525
mc_ID_2,Tip Top Savings,Supermarket,8400000.0,3192000.0,16.7163,12.3555,False,30000.0,2811.99,2049.72,...,WA,Carrington,Frozen Desserts,Frozen Foods,0,0,0,0,0,4.3608
mc_ID_3,Price Cutters,NaN,4600000.0,1518000.0,24.1296,NaN,True,10000.0,2147.17,NaN,...,BC,PigTail,Frozen Desserts,Frozen Foods,1,1,1,1,1,NaN
mc_ID_4,Money Savers,Supermarket,6810000.0,3200700.0,17.8792,16.4256,False,50000.0,3685.30,2265.90,...,WA,Moms,Meat,Deli,0,0,1,0,1,1.4536


In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12956 entries, mc_ID_0 to mc_ID_12955
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Promotion Name             11665 non-null  object 
 1   Store Kind                 11683 non-null  object 
 2   Store Sales                12955 non-null  float64
 3   Store Cost                 12956 non-null  float64
 4   Gross Weight               11014 non-null  float64
 5   Net Weight                 11018 non-null  float64
 6   Is Recyclable?             12956 non-null  bool   
 7   Min. Person Yearly Income  12952 non-null  float64
 8   Store Area                 11663 non-null  float64
 9   Grocery Area               11658 non-null  float64
 10  Frozen Area                11656 non-null  float64
 11  Meat Area                  11688 non-null  float64
 12  Cost                       12953 non-null  float64
 13  Marriage                   12956 non-nu

In [24]:
df3.head()

,Unnamed: 0,Person%20Description,Place%20Code,Customer%20Order,Additional%20Features%20in%20market,Promotion%20Name,Store%20Kind,Store%20Sales,Store%20Cost,Gross%20Weight,Net%20Weight,Weights%20Data,Is%20Recyclable?,Yearly%20Income,Store%20Area,Grocery%20Area,Frozen%20Area,Meat%20Area,Cost
0,mc_ID_0,"Single Female with two children, education: pa...",T02ma_WA,"Meat from Deli department, Ordered Brand : Red...","['Coffee Bar', 'Florist', 'Ready Food', 'Bar F...",Sale : Double Down,Deluxe,7.12 Millions,2.5632 Millions,23.2575,20.3503,"{'Gross Weight': 23.2575, 'Net Weight': 20.350...",yes,90K+,3145.51,2056.79,654.13,436.09,500.7202
1,mc_ID_1,"Single Female with five children, education: p...",M10da_YU,"Specialty from Produce department, Ordered Bra...","['Coffee Bar', 'Florist', 'Bar For Salad', 'Vi...",GLD,Deluxe,14.72 Millions,7.0656 Millions,16.7163,12.3555,"{'Gross Weight': 16.7163, 'Net Weight': 12.355...",yes,30K+,2856.68,1871.16,595.93,395.51,484.1411
2,mc_ID_2,"Married Male with three children, education: h...",T02ma_WA,"Paper Products from Household department, Orde...","['Coffee Bar', 'Bar For Salad', 'Ready Food', ...",Promo Big,Deluxe,4.98 Millions,2.4402 Millions,10.5531,7.6459,"{'Gross Weight': 10.5531, 'Net Weight': 7.6459...",no,30K+,3140.99,2055.29,654.13,436.09,267.3576
3,mc_ID_3,"Married Female with five children, education: ...",H11go_ZA,"Snack Foods from Snack Foods department, Order...","['Video Store', 'Coffee Bar', 'Ready Food', 'F...",Price Destroyers,NaN,5.24 Millions,2.0436 Millions,23.6936,NaN,"{'Gross Weight': 23.6936, 'Net Weight': 20.641...",no,10K+,2841.35,2038.11,481.98,322.22,777.2826
4,mc_ID_4,"Married Female with No children, education: pa...",O09ba_VE,"Canned Soup from Canned Foods department, Orde...","['Ready Food', 'Bar For Salad']",Save Price,Supermarket,8.84 Millions,3.4476 Millions,13.6056,10.6839,"{'Gross Weight': 13.6056, 'Net Weight': 10.683...",no,110K+,3236.33,2448.37,472.27,313.87,406.4756


In [25]:
df3.columns = df3.columns.str.replace('%20', ' ')
df3 = pre.set_index(df3)
df3 = pre.split_person_description(df3)
df3 = pre.split_place_code(df3)
df3 = pre.split_customer_order(df3)
df3 = pre.encode_market_features(df3)
df3 = pre.transform_cost_sales(df3)
df3 = pre.transform_income(df3)
df3 = pre.calculate_package_weight(df3)
#df3 = transform_columns_type(df3)
df3 = df3.drop(columns=["", "Weights Data"])
df3 = pre.encode_columns(df3)
df3.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Is Recyclable?,Store Area,Grocery Area,Frozen Area,...,Order Brand,Order,Department,Coffee Bar,Florist,Bar For Salad,Video Store,Ready Food,Min. Person Yearly Income,Package Weight
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Sale : Double Down,Deluxe,7120000.0,2563200.0,23.2575,20.3503,True,3145.51,2056.79,654.13,...,Red Spade,Meat,Deli,1,1,1,0,1,90000.0,2.9072
mc_ID_1,GLD,Deluxe,14720000.0,7065600.0,16.7163,12.3555,True,2856.68,1871.16,595.93,...,Tell Tale,Specialty,Produce,1,1,1,1,1,30000.0,4.3608
mc_ID_2,Promo Big,Deluxe,4980000.0,2440200.0,10.5531,7.6459,False,3140.99,2055.29,654.13,...,High Quality,Paper Products,Household,1,1,1,0,1,30000.0,2.9072
mc_ID_3,Price Destroyers,NaN,5240000.0,2043600.0,23.6936,NaN,False,2841.35,2038.11,481.98,...,Nationeel,Snack Foods,Snack Foods,1,1,1,1,1,10000.0,NaN
mc_ID_4,Save Price,Supermarket,8840000.0,3447600.0,13.6056,10.6839,False,3236.33,2448.37,472.27,...,Better,Canned Soup,Canned Foods,0,0,1,0,1,110000.0,2.9217


In [26]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6466 entries, mc_ID_0 to mc_ID_6465
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Promotion Name             4839 non-null   object 
 1   Store Kind                 4840 non-null   object 
 2   Store Sales                6452 non-null   float64
 3   Store Cost                 6459 non-null   float64
 4   Gross Weight               5491 non-null   float64
 5   Net Weight                 5511 non-null   float64
 6   Is Recyclable?             6466 non-null   bool   
 7   Store Area                 6151 non-null   float64
 8   Grocery Area               6138 non-null   float64
 9   Frozen Area                6147 non-null   float64
 10  Meat Area                  6133 non-null   float64
 11  Cost                       6462 non-null   float64
 12  Marriage                   6466 non-null   object 
 13  Gender                     6466 non-null 

In [27]:
df = pd.concat([df1, df2, df3])
df.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,...,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.0,4292400.0,True,2842.23,2037.64,481.98,323.00,602.7575,...,Household,1,1,1,1,1,28.1997,26.6008,1.5990,10000.0
mc_ID_1,Budget Bargains,Supermarket,6360000.0,1971600.0,False,2814.95,2049.72,457.36,NaN,708.6650,...,Snack Foods,0,0,0,0,0,16.5710,14.9720,1.5990,50000.0
mc_ID_2,Shelf Emptiers,Supermarket,10860000.0,4452600.0,True,2192.32,1322.21,523.32,348.85,564.2647,...,Periodicals,1,0,0,0,0,28.6358,27.1822,1.4536,30000.0
mc_ID_3,Savings Galore,NaN,1980000.0,673200.0,True,1974.73,NaN,440.92,293.95,724.5119,...,Dairy,0,1,0,0,0,14.2161,11.2944,2.9217,30000.0
mc_ID_4,Sale Winners,Deluxe,11560000.0,4970800.0,False,2862.30,1872.19,593.93,395.95,519.7574,...,Produce,1,1,1,1,1,12.6172,9.7100,2.9072,50000.0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38862 entries, mc_ID_0 to mc_ID_6465
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Promotion Name             33034 non-null  object 
 1   Store Kind                 33053 non-null  object 
 2   Store Sales                38847 non-null  float64
 3   Store Cost                 38855 non-null  float64
 4   Is Recyclable?             38862 non-null  bool   
 5   Store Area                 35296 non-null  float64
 6   Grocery Area               35255 non-null  float64
 7   Frozen Area                35309 non-null  float64
 8   Meat Area                  35313 non-null  float64
 9   Cost                       38831 non-null  float64
 10  Marriage                   38862 non-null  object 
 11  Gender                     38862 non-null  object 
 12  Children                   38862 non-null  int32  
 13  Degree                     38862 non-nul

In [29]:
df.to_csv("data/half_cleaned.csv")

In [30]:
df = pre.fill_nulls(df)
df.isna().sum()

Promotion Name               5828
Store Kind                   5809
Store Sales                     0
Store Cost                      0
Is Recyclable?                  0
Store Area                      0
Grocery Area                    0
Frozen Area                     0
Meat Area                       0
Cost                            0
Marriage                        0
Gender                          0
Children                        0
Degree                          0
Work                            0
Store Code                      0
Country ISO2                    0
Order Brand                     2
Order                           2
Department                      2
Florist                         0
Coffee Bar                      0
Bar For Salad                   0
Video Store                     0
Ready Food                      0
Gross Weight                    0
Net Weight                      0
Package Weight                  0
Min. Person Yearly Income       0
dtype: int64

In [31]:
df.dropna(inplace=True)
df.isna().sum()

Promotion Name               0
Store Kind                   0
Store Sales                  0
Store Cost                   0
Is Recyclable?               0
Store Area                   0
Grocery Area                 0
Frozen Area                  0
Meat Area                    0
Cost                         0
Marriage                     0
Gender                       0
Children                     0
Degree                       0
Work                         0
Store Code                   0
Country ISO2                 0
Order Brand                  0
Order                        0
Department                   0
Florist                      0
Coffee Bar                   0
Bar For Salad                0
Video Store                  0
Ready Food                   0
Gross Weight                 0
Net Weight                   0
Package Weight               0
Min. Person Yearly Income    0
dtype: int64

In [32]:
df.to_csv("data/cleaned_train.csv")

In [33]:
test = pd.read_csv("data/test.csv")
test.head()

,Unnamed: 0,Person Description,Place Code,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area
0,0,"Single Male with four children, education: par...",B06ls_CA,"Candy from Snacks department, Ordered Brand : ...","['Florist', 'Ready Food', 'Video Store', 'Bar ...",Fantastic Discounts,Gourmet,11.76 Millions,4.704 Millions,31.8337,28.7811,3.0525,yes,50K+,2201.06,1424.85,465.54,308.73
1,1,"Married Female with three children, education:...",S01em_OR,"Vegetables from Frozen Foods department, Order...","['Florist', 'Coffee Bar', 'Ready Food', 'Bar F...",Bag Stuffer,Deluxe,2.16 Millions,0.6696 Millions,29.9440,27.0368,2.9072,no,70K+,2577.16,1735.17,505.07,336.59
2,2,"Single Female with three children, education: ...",H11go_ZA,"Dairy from Dairy department, Ordered Brand : Club","['Bar For Salad', 'Video Store', 'Ready Food',...",Pick Your Savings,Deluxe,1.83 Millions,0.8235 Millions,29.2172,26.3100,2.9072,no,130K+,2837.58,2038.11,481.98,321.26
3,3,"Single Male with three children, education: pa...",M10da_YU,"Frozen Desserts from Frozen Foods department, ...","['Coffee Bar', 'Video Store', 'Bar For Salad',...",Price Winners,Deluxe,8.82 Millions,4.41 Millions,28.0543,25.0018,3.0525,no,10K+,2859.04,1871.16,593.93,394.58
4,4,"Single Male with two children, education: high...",L05es_CA,"Hot Beverages from Beverages department, Order...",['Florist'],Dollar Days,Supermarket,4.32 Millions,1.9872 Millions,23.5482,20.6410,2.9072,no,30K+,2193.97,1320.15,523.32,348.85


In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19942 entries, 0 to 19941
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     19942 non-null  int64  
 1   Person Description             19942 non-null  object 
 2   Place Code                     19942 non-null  object 
 3   Customer Order                 19942 non-null  object 
 4   Additional Features in market  19942 non-null  object 
 5   Promotion Name                 19916 non-null  object 
 6   Store Kind                     19914 non-null  object 
 7   Store Sales                    19942 non-null  object 
 8   Store Cost                     19942 non-null  object 
 9   Gross Weight                   18959 non-null  float64
 10  Net Weight                     18924 non-null  float64
 11  Package Weight                 19942 non-null  float64
 12  Is Recyclable?                 19942 non-null 

In [35]:
test = pre.set_index(test)
test = pre.split_person_description(test)
test = pre.split_place_code(test)
test = pre.split_customer_order(test)
test = pre.encode_market_features(test)
test = pre.transform_cost_sales(test)
test = pre.transform_income(test)
#test = pre.transform_columns_type(test)
test = test.drop(columns="")
test = pre.fill_nulls(test)
test = pre.encode_columns(test)
test.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Store Area,Grocery Area,...,Country ISO2,Order Brand,Order,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Min. Person Yearly Income
id,,,,,,,,,,,,,,,,,,,,,
0,Fantastic Discounts,Gourmet,11760000.0,4704000.0,31.8337,28.7811,3.0525,True,2201.06,1424.85,...,CA,Gulf Coast,Candy,Snacks,1,1,1,1,1,50000.0
1,Bag Stuffer,Deluxe,2160000.0,669600.0,29.9440,27.0368,2.9072,False,2577.16,1735.17,...,OR,Golden,Vegetables,Frozen Foods,1,1,1,1,1,70000.0
2,Pick Your Savings,Deluxe,1830000.0,823500.0,29.2172,26.3100,2.9072,False,2837.58,2038.11,...,ZA,Club,Dairy,Dairy,1,1,1,1,1,130000.0
3,Price Winners,Deluxe,8820000.0,4410000.0,28.0543,25.0018,3.0525,False,2859.04,1871.16,...,YU,Imagine,Frozen Desserts,Frozen Foods,1,1,1,1,1,10000.0
4,Dollar Days,Supermarket,4320000.0,1987200.0,23.5482,20.6410,2.9072,False,2193.97,1320.15,...,CA,BBB Best,Hot Beverages,Beverages,1,0,0,0,0,30000.0


In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19942 entries, 0 to 19941
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Promotion Name             19942 non-null  object 
 1   Store Kind                 19942 non-null  object 
 2   Store Sales                19942 non-null  float64
 3   Store Cost                 19942 non-null  float64
 4   Gross Weight               19942 non-null  float64
 5   Net Weight                 19942 non-null  float64
 6   Package Weight             19942 non-null  float64
 7   Is Recyclable?             19942 non-null  bool   
 8   Store Area                 19942 non-null  float64
 9   Grocery Area               19942 non-null  float64
 10  Frozen Area                19942 non-null  float64
 11  Meat Area                  19942 non-null  float64
 12  Marriage                   19942 non-null  object 
 13  Gender                     19942 non-null  obj

In [37]:
test.isna().sum()

Promotion Name               0
Store Kind                   0
Store Sales                  0
Store Cost                   0
Gross Weight                 0
Net Weight                   0
Package Weight               0
Is Recyclable?               0
Store Area                   0
Grocery Area                 0
Frozen Area                  0
Meat Area                    0
Marriage                     0
Gender                       0
Children                     0
Degree                       0
Work                         0
Store Code                   0
Country ISO2                 0
Order Brand                  0
Order                        0
Department                   0
Florist                      0
Coffee Bar                   0
Bar For Salad                0
Video Store                  0
Ready Food                   0
Min. Person Yearly Income    0
dtype: int64

In [38]:
test = test[df.drop(columns="Cost").columns]
test.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,Meat Area,Marriage,...,Department,Florist,Coffee Bar,Bar For Salad,Video Store,Ready Food,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
id,,,,,,,,,,,,,,,,,,,,,
0,Fantastic Discounts,Gourmet,11760000.0,4704000.0,True,2201.06,1424.85,465.54,308.73,Single,...,Snacks,1,1,1,1,1,31.8337,28.7811,3.0525,50000.0
1,Bag Stuffer,Deluxe,2160000.0,669600.0,False,2577.16,1735.17,505.07,336.59,Married,...,Frozen Foods,1,1,1,1,1,29.9440,27.0368,2.9072,70000.0
2,Pick Your Savings,Deluxe,1830000.0,823500.0,False,2837.58,2038.11,481.98,321.26,Single,...,Dairy,1,1,1,1,1,29.2172,26.3100,2.9072,130000.0
3,Price Winners,Deluxe,8820000.0,4410000.0,False,2859.04,1871.16,593.93,394.58,Single,...,Frozen Foods,1,1,1,1,1,28.0543,25.0018,3.0525,10000.0
4,Dollar Days,Supermarket,4320000.0,1987200.0,False,2193.97,1320.15,523.32,348.85,Single,...,Beverages,1,0,0,0,0,23.5482,20.6410,2.9072,30000.0


In [39]:
test.to_csv("data/cleaned_test.csv")